# Process data config

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append('..')

from os import environ
import pandas as pd
from indicators import indicators
from datetime import timedelta
from tqdm.auto import tqdm
from config.config import ConfigFactory

class CFG:
    cls_target_ratio = 1.021

# Load data and add indicators

In [38]:
# Set environment variable
environ["ENV"] = "1h_4h"

# Get configs
configs = ConfigFactory.factory(environ).configs

def get_file(ticker):
    ''' Find files buy ticker names, file names can be in different formats '''
    try:
        tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker}_1h.pkl')
        tmp_df_4h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker}_4h.pkl')
    except FileNotFoundError:
        pass
    else:
        return tmp_df_1h, tmp_df_4h
    
    try:
        tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}_1h.pkl')
        tmp_df_4h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}_4h.pkl')
    except FileNotFoundError:
        pass
    else:
        return tmp_df_1h, tmp_df_4h
    
    try:
        tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}-SWAP_1h.pkl')
        tmp_df_4h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}-SWAP_4h.pkl')
    except FileNotFoundError:
        pass
    else:
        return tmp_df_1h, tmp_df_4h
    
    return None, None

def add_indicators(df, ttype, configs):
    # add RSI
    rsi = indicators.RSI(ttype, configs)
    df = rsi.get_indicator(df, '', '', 0)
    # add RSI
    stoch = indicators.STOCH(ttype, configs)
    df = stoch.get_indicator(df, '', '', 0)
    # add Trend
    trend = indicators.Trend(ttype, configs)
    df = trend.get_indicator(df, '', '', 0)
    # add MACD
    macd = indicators.MACD(ttype, configs)
    df = macd.get_indicator(df, '', '', 0)
    # add ATR
    atr = indicators.ATR(ttype, configs)
    df = atr.get_indicator(df, '', '', 0)
    # add SMA
    # sma = indicators.SMA(ttype, configs)
    # df = sma.get_indicator(df, '', '', 0)
    return df

def create_train_df(df, ttype, configs, target_offset, first, last, step):
    ''' Create train dataset from signal statistics and ticker candle data'''
    train_df = pd.DataFrame()
    tickers = df['ticker'].unique()
    
    for ticker in tqdm(tickers):
        # get signals with current ticker
        signal_df = df[df['ticker'] == ticker]
        times = signal_df['time']
        
        # load candle history of this ticker
        tmp_df_1h, tmp_df_4h = get_file(ticker)

        # add indicators 
        try:
            tmp_df_1h = add_indicators(tmp_df_1h, ttype, configs)
        except TypeError:
            continue

        # add historical data for current ticker
        for i, t in enumerate(times.to_list()):
            pass_cycle = False
            pattern = signal_df.iloc[i, signal_df.columns.get_loc('pattern')]
            row = tmp_df_1h.loc[tmp_df_1h['time'] == t, :].reset_index(drop=True)
            
            for i in range(first, last + step, step):
                time_prev = t + timedelta(hours= -i)
                try:
                    row_tmp = tmp_df_1h.loc[tmp_df_1h['time'] == time_prev, :].reset_index(drop=True)
                    row_tmp.columns = [c + f'_prev_{i}' for c in row_tmp.columns]
                except IndexError:
                    pass_cycle = True
                    break
                row = pd.concat([row, row_tmp.iloc[:,1:]], axis=1)
                row['ticker'] = ticker
                row['pattern'] = pattern
                
            if pass_cycle:
                continue

            row['target'] = 0
            
            if row['pattern'].values == 'STOCH_RSI':
                if ttype == 'buy':
                    row['ttype'] = 'sell'
                else:
                    row['ttype'] = 'buy'
            else:
                row['ttype'] = ttype
            
            # If ttype = buy and during the selected period high price was higher than close_price * target_ratio
            # and earlier low price wasn't lower than close_price / target_ratio, than target is True, else target is False.
            # Similarly for ttype = sell 
            close_price = tmp_df_1h.loc[tmp_df_1h['time'] == t, 'close'].values
            
            for i in range(1, target_offset + 1):
                time_next = t + timedelta(hours=i)
                target_buy = tmp_df_1h.loc[tmp_df_1h['time'] == time_next, 'high'].reset_index(drop=True)
                target_sell = tmp_df_1h.loc[tmp_df_1h['time'] == time_next, 'low'].reset_index(drop=True)

                try:
                    target_buy = target_buy > close_price * CFG.cls_target_ratio
                    target_sell = target_sell < close_price / CFG.cls_target_ratio
                except ValueError:
                    pass_cycle = True
                    break
                
                try:
                    if (row['ttype'].values == 'buy' and target_sell[0]) or (row['ttype'].values == 'sell' and target_buy[0]):
                        break
                    elif (row['ttype'].values == 'buy' and target_buy[0]) or (row['ttype'].values == 'sell' and target_sell[0]):
                        row['target'] = 1
                        break
                except KeyError:
                    pass_cycle = True
                    break
            
            if pass_cycle:
                continue

            # add data to the dataset
            if train_df.shape[0] == 0:
                train_df = row
            else:
                train_df = pd.concat([train_df, row])
    
    return train_df

# for how long time (in hours) we want to predict
target_offset = 48
# first previous data point to collect for model training (value represents number of hours before signal point)
first = 4
# last previous data point to collect for model training (value represents number of hours before signal point)
last = 96
# step of previous data points collecting (total number of points to collect is (last - first + step) / step)
step = 4

# Buy
# dataset with the signal statistics
df = pd.read_pickle('signal_stat/buy_stat_1h.pkl')
# dataset for model train
train_buy = create_train_df(df, 'buy', configs, target_offset, first, last, step)
train_buy = train_buy.dropna()

# Sell
# dataset with the signal statistics
df = pd.read_pickle('signal_stat/sell_stat_1h.pkl')
# dataset for model train
train_sell = create_train_df(df, 'sell', configs, target_offset, first, last, step)
train_sell = train_sell.dropna()

train_buy = pd.concat([train_buy, train_sell[train_sell['ttype'] == 'buy']]).sort_values('time').reset_index(drop=True)
train_sell = pd.concat([train_sell, train_buy[train_buy['ttype'] == 'sell']]).sort_values('time').reset_index(drop=True)

train_buy = train_buy[train_buy['ttype'] == 'buy']
train_sell = train_sell[train_sell['ttype'] == 'sell']

train_buy.to_pickle(f'signal_stat/train_buy_{last}.pkl')
train_sell.to_pickle(f'signal_stat/train_sell_{last}.pkl')

display(df.head())
display(df.shape)


  0%|          | 0/362 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Check pattern/target distribution

In [15]:
train_buy[['target', 'pattern']].value_counts()

target  pattern      
0       Pattern_Trend    1566
1       Pattern_Trend    1485
        MACD              913
0       MACD              566
        STOCH_RSI         564
1       STOCH_RSI         530
Name: count, dtype: int64

In [16]:
train_sell[['target', 'pattern']].value_counts()

target  pattern      
1       MACD             1852
0       MACD             1157
1       Pattern_Trend     927
0       Pattern_Trend     919
1       STOCH_RSI         587
0       STOCH_RSI         514
Name: count, dtype: int64

# Check target correctness

In [41]:
train_buy['target'].value_counts()

target
1    2928
0    2696
Name: count, dtype: int64

In [42]:
train_sell['target'].value_counts()

target
1    3366
0    2590
Name: count, dtype: int64

In [57]:
i = 654

x = train_sell.loc[train_sell.target == 0, ['ticker', 'ttype', 'pattern', 'time', 'close', 'target']]
y = x.iloc[i]
low_price, high_price = y['close'] / CFG.cls_target_ratio, y['close'] * CFG.cls_target_ratio,
print(y['ticker'], y['time'], y['ttype'])

tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{y["ticker"]}_1h.pkl')
# tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{y["ticker"][:-4]}-{y["ticker"][-4:]}_1h.pkl')
# tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{y["ticker"][:-4]}-{y["ticker"][-4:]}-SWAP_4h.pkl')

tmp_df_1h['low_price'] = low_price
tmp_df_1h['high_price'] = high_price
idx = tmp_df_1h[tmp_df_1h['time'] == y['time']].index[0]

tmp_df_1h = tmp_df_1h.iloc[idx:idx+target_offset+1][['time', 'close', 'high', 'high_price', 'low', 'low_price']]

if y['ttype'] == 'buy':
    tmp_df_1h['signal'] = tmp_df_1h['high'] > tmp_df_1h['high_price']
    tmp_df_1h['anti_signal'] = tmp_df_1h['low'] < tmp_df_1h['low_price']
else:
    tmp_df_1h['signal'] = tmp_df_1h['low'] < tmp_df_1h['low_price']
    tmp_df_1h['anti_signal'] = tmp_df_1h['high'] > tmp_df_1h['high_price']

tmp_df_1h

STGUSDT 2023-03-15 19:00:00 sell


,time,close,high,high_price,low,low_price,signal,anti_signal
2039,2023-03-15 19:00:00,0.7094,0.7308,0.724297,0.7031,0.694809,False,True
2040,2023-03-15 20:00:00,0.7268,0.7270,0.724297,0.7078,0.694809,False,True
2041,2023-03-15 21:00:00,0.7427,0.7431,0.724297,0.7236,0.694809,False,True
2042,2023-03-15 22:00:00,0.7378,0.7467,0.724297,0.7301,0.694809,False,True
2043,2023-03-15 23:00:00,0.7385,0.7477,0.724297,0.7333,0.694809,False,True
2044,2023-03-16 00:00:00,0.7362,0.7411,0.724297,0.7338,0.694809,False,True
2045,2023-03-16 01:00:00,0.7278,0.7418,0.724297,0.7252,0.694809,False,True
2046,2023-03-16 02:00:00,0.7287,0.7312,0.724297,0.7233,0.694809,False,True
2047,2023-03-16 03:00:00,0.7209,0.7397,0.724297,0.7195,0.694809,False,True
2048,2023-03-16 04:00:00,0.7252,0.7321,0.724297,0.7200,0.694809,False,True
